In [30]:
from datetime import datetime

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [ ]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [4]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [5]:
client.create_experiment(name="my-cool-new-experiment")

'2'

Runs can be filtered:

In [14]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string='metrics.rmse < 6.35',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=['metrics.rmse ASC']
)

In [15]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: c1e4375288114b19b25a1981c3cc4f54, rmse: 6.3040
run id: 1c998e0a35e845f6bacb2c70e0c49d1e, rmse: 6.3040
run id: bfb3d660a8c3499780a84872b6fce95a, rmse: 6.3040
run id: 34a10ddf92d74d71b688bd4c1abe4b88, rmse: 6.3107


Register models from Python:

In [18]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [22]:
run_id = '34a10ddf92d74d71b688bd4c1abe4b88'
model_uri = f'runs:/{run_id}/model'
mlflow.register_model(model_uri=model_uri, name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/30 18:17:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1653927436704, current_stage='None', description=None, last_updated_timestamp=1653927436704, name='nyc-taxi-regressor', run_id='34a10ddf92d74d71b688bd4c1abe4b88', run_link=None, source='./mlruns/1/34a10ddf92d74d71b688bd4c1abe4b88/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

Transition models from Python 

In [26]:
# See which version of the model is in which stage:
model_name = "nyc-taxi-regressor"
lastest_versions = client.get_latest_versions(name=model_name)

for version in lastest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


In [29]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage, 
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653927436704, current_stage='Staging', description=None, last_updated_timestamp=1653927863743, name='nyc-taxi-regressor', run_id='34a10ddf92d74d71b688bd4c1abe4b88', run_link=None, source='./mlruns/1/34a10ddf92d74d71b688bd4c1abe4b88/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

Update the model description

In [31]:
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {datetime.now()}."
)

<ModelVersion: creation_timestamp=1653927436704, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2022-05-30 18:31:03.151736.', last_updated_timestamp=1653928263153, name='nyc-taxi-regressor', run_id='34a10ddf92d74d71b688bd4c1abe4b88', run_link=None, source='./mlruns/1/34a10ddf92d74d71b688bd4c1abe4b88/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>